# Presentation

In this notebook, we are making a few graphs that will help us understand the data.

In [1]:
import pandas as pd
import sys

%matplotlib inline

sys.path.append('./src/')
from plots import *

In [2]:
data = pd.read_csv("data/slo_weather_history.csv")
data.head()

,date,dew_point_f_avg,dew_point_f_high,dew_point_f_low,events,humidity_%_avg,humidity_%_high,humidity_%_low,precip_in_sum,sea_level_press_in_avg,...,sea_level_press_in_low,temp_f_avg,temp_f_high,temp_f_low,visibility_mi_avg,visibility_mi_high,visibility_mi_low,wind_gust_mph_high,wind_mph_avg,wind_mph_high
0,2012-1-1,44.0,50.0,34.0,Fog,80.0,100.0,25.0,0.00,30.15,...,30.08,56.0,73.0,39.0,6.0,10.0,0.0,0.0,1.0,8.0
1,2012-1-2,47.0,52.0,43.0,Fog,93.0,100.0,63.0,0.00,30.23,...,30.19,52.0,63.0,42.0,4.0,10.0,0.0,0.0,3.0,14.0
2,2012-1-3,43.0,50.0,37.0,Fog,85.0,100.0,32.0,0.01,30.24,...,30.17,58.0,77.0,39.0,6.0,10.0,0.0,0.0,2.0,10.0
3,2012-1-4,42.0,47.0,37.0,,69.0,96.0,33.0,0.00,30.24,...,30.20,56.0,73.0,39.0,10.0,10.0,8.0,0.0,1.0,9.0
4,2012-1-5,42.0,51.0,36.0,,66.0,93.0,23.0,0.00,30.15,...,30.09,60.0,78.0,42.0,10.0,10.0,7.0,22.0,4.0,18.0


In [3]:
# plot_all_data(data)

In [4]:
# simple_time_series_forecast_persistence_rmse(data)

In [5]:
# simple_time_series_forecast_expanding_window_rmse(data)

In [6]:
# simple_time_series_forecast_rolling_window_rmse(data)

In [7]:
# out_of_sample_forecast_arima(data)

In [8]:
fixed_autoregression_forecast(data)

Day 1 -- Forecast: 52.116721, Actual: 57.000000
Day 2 -- Forecast: 51.344425, Actual: 54.000000
Day 3 -- Forecast: 51.397121, Actual: 48.000000
Day 4 -- Forecast: 51.601338, Actual: 53.000000
Day 5 -- Forecast: 51.568685, Actual: 53.000000
Day 6 -- Forecast: 51.614797, Actual: 52.000000
Day 7 -- Forecast: 51.461988, Actual: 51.000000

Test RMSE: 2.586


In [9]:
rolling_autoregression_forecast(data)

Day 1 -- Forecast: 52.116721, Actual: 57.000000
Day 2 -- Forecast: 54.648137, Actual: 54.000000
Day 3 -- Forecast: 53.234415, Actual: 48.000000
Day 4 -- Forecast: 49.439777, Actual: 53.000000
Day 5 -- Forecast: 52.455022, Actual: 53.000000
Day 6 -- Forecast: 52.781134, Actual: 52.000000
Day 7 -- Forecast: 51.910638, Actual: 51.000000

Test RMSE: 3.072
